In [29]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [27]:
def clean_text(txt_in):
    import re
    clean = re.sub('[^A-Za-z0-9]+', " ", txt_in).strip()
    return clean
def rem_sw(var):
    from nltk.corpus import stopwords
    sw = set(stopwords.words("English"))
    my_test = [word for word in var.split() if word not in sw]
    my_test = ' '.join(my_test)
    return my_test
def updown(x):
    if x >0:
        return 1
    else:
        return 0

In [3]:
#Price dataframe calculate percentage difference
prices = pd.read_csv('GM.csv')
prices['diff'] = (prices['Close'] - prices['Open'])/prices['Open']
prices['date'] = pd.to_datetime(prices['Date']).dt.date

In [4]:
#filter tweets for market hours
tweets = pd.read_csv('tweets2.csv')[['created_at','full_text']]
tweets['dt'] = pd.to_datetime(tweets['created_at'])
tweets.set_index('dt', inplace=True)
tweets = tweets.between_time('09:30', '16:00')
tweets.index = pd.to_datetime(tweets.index)
fil = tweets.between_time('09:30:00', '16:00:00')
tweets['date'] = tweets.index.date

In [5]:
#basic cleaning
tweets['tweet'] = tweets['full_text'].apply(clean_text).apply(rem_sw)

In [7]:
#merge with price data
clean = tweets[['date','tweet']].merge(prices, on='date')

In [8]:
#VADER sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [9]:
clean['compound'] = [analyzer.polarity_scores(x)['compound'] for x in clean['tweet']]
clean['neg'] = [analyzer.polarity_scores(x)['neg'] for x in clean['tweet']]
clean['neu'] = [analyzer.polarity_scores(x)['neu'] for x in clean['tweet']]
clean['pos'] = [analyzer.polarity_scores(x)['pos'] for x in clean['tweet']]

In [10]:
x = ['compound','neg','neu','pos']

In [13]:
result = sm.OLS(clean['diff'], clean[x]).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   diff   R-squared (uncentered):                   0.010
Model:                            OLS   Adj. R-squared (uncentered):             -0.009
Method:                 Least Squares   F-statistic:                             0.5178
Date:                Sat, 04 Dec 2021   Prob (F-statistic):                       0.723
Time:                        21:59:53   Log-Likelihood:                          560.49
No. Observations:                 205   AIC:                                     -1113.
Df Residuals:                     201   BIC:                                     -1100.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
compound      -0.0049      0.007     -0.674      0.501      -0.019       0.009
neg            0.0132      0.021      0.622      0.534      -0.029       0.055
neu            0.0016      0.003      0.546      0.585      -0.004       0.007
pos            0.0080      0.013      0.607      0.545      -0.018       0.034
==============================================================================
Omnibus:                       11.729   Durbin-Watson:                   1.820
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               20.998
Skew:                           0.270   Prob(JB):                     2.76e-05
Kurtosis:                       4.472   Cond. No.                         20.8
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
#Logistic regression for price increase
clean['up'] = clean['diff'].apply(updown)
logreg = LogisticRegression().fit(clean[x], clean['up'])

In [33]:
confusion_matrix(clean['up'], logreg.predict(clean[x]))

array([[100,   7],
       [ 87,  11]])